<a href="https://colab.research.google.com/github/CALDISS-AAU/sdsphd21/blob/master/notebooks/NLP_Semantic_search_caldissphd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('scopus.csv')

In [ ]:
data

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,Page count,Cited by,DOI,Link,Abstract,Document Type,Publication Stage,Open Access,Source,EID
0,"Tseng F.-C., Huang T.-L., Pham T.T.L., Cheng T...",36146359200;57203386123;57200393414;1330259380...,How does media richness foster online gamer lo...,2022,International Journal of Information Management,62,NaN,102439,NaN,NaN,NaN,NaN,10.1016/j.ijinfomgt.2021.102439,https://www.scopus.com/inward/record.uri?eid=2...,"The online game market is highly popular, resu...",Article,Final,"All Open Access, Bronze",Scopus,2-s2.0-85117119139
1,"Nelci J., Salloum S.A., Shaalan K.",57231400500;57195670894;6507669702;,An Overview of Sentiment Analysis with Dialect...,2022,Lecture Notes in Networks and Systems,299,NaN,NaN,3.0,12,NaN,NaN,10.1007/978-3-030-82616-1_1,https://www.scopus.com/inward/record.uri?eid=2...,Sentiment analysis (SA) is one of the most eme...,Conference Paper,Final,NaN,Scopus,2-s2.0-85113452180
2,"Kietzka G.J., Pryke J.S., Gaigher R., Samways ...",56644914500;16319927900;35752984700;7004518089;,32 years of essential management to retain val...,2021,Urban Ecosystems,24,6,NaN,1295.0,1304,NaN,NaN,10.1007/s11252-021-01115-5,https://www.scopus.com/inward/record.uri?eid=2...,Pond construction in urban areas can mitigate ...,Article,Final,NaN,Scopus,2-s2.0-85118034872
3,"Yaylaci S., Ulman Y.I., Vatansever K., Senyure...",25653347000;8694295000;23500265800;57216939375...,"Integrating patient management, reflective pra...",2021,BMC Medical Education,21,1,536,NaN,NaN,NaN,NaN,10.1186/s12909-021-02970-8,https://www.scopus.com/inward/record.uri?eid=2...,"Background: Integration of clinical skills, et...",Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85117711247
4,"Lampa E., Sonnentheil B., Tökés A., Warner G.",57211752386;57290992900;57289942900;57211748870;,What has the COVID-19 pandemic taught us about...,2021,Research Involvement and Engagement,7,1,73,NaN,NaN,NaN,NaN,10.1186/s40900-021-00315-9,https://www.scopus.com/inward/record.uri?eid=2...,Background: During the COVID-19 pandemic many ...,Note,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85116835413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,Sheffield J.,7102636616;,The effects of bargaining orientation and comm...,1989,Conference on Human Factors in Computing Syste...,NaN,NaN,NaN,43.0,48,NaN,11.0,10.1145/67449.67458,https://www.scopus.com/inward/record.uri?eid=2...,Pairs of subjects with either a competitive or...,Conference Paper,Final,NaN,Scopus,2-s2.0-84901685084
1329,"Trevino L.K., Lengel R.H., Daft R.L.",7003822350;25632312800;6603220782;,"Media Symbolism, Media Richness, and Media Cho...",1987,Communication Research,14,5,NaN,553.0,574,NaN,546.0,10.1177/009365087014005006,https://www.scopus.com/inward/record.uri?eid=2...,Symbolic interactionism is presented as a theo...,Article,Final,NaN,Scopus,2-s2.0-84965761571
1330,"Daft R.L., Lengel R.H., Trevino L.K.",6603220782;25632312800;7003822350;,"Message equivocality, media selection, and man...",1987,MIS Quarterly: Management Information Systems,11,3,NaN,355.0,366,NaN,1419.0,10.2307/248682,https://www.scopus.com/inward/record.uri?eid=2...,A field study of middle- and upper-level manag...,Article,Final,NaN,Scopus,2-s2.0-55249120307
1331,Lakin F.,6506936572;,A performing medium for working group graphics,1986,Proceedings of the ACM Conference on Computer ...,NaN,NaN,NaN,255.0,266,NaN,8.0,10.1145/637069.637105,https://www.scopus.com/inward/record.uri?eid=2...,Writing and drawing together on a common displ...,Conference Paper,Final,NaN,Scopus,2-s2.0-85024576240


In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
# Let's fire up spacy

import spacy

# and load the small english language model. Large models can be downloaded for many languages.
nlp = spacy.load("en_core_web_sm")

# find more models for other languages here: https://spacy.io/models/

In [ ]:
# Let's tokenize all abstracts and get a filtered list of tokens
tokenlist = []
for doc in nlp.pipe(data['Abstract'], disable=["parser", "ner"]):
  tokens =[tok.text.lower() for tok in doc if tok.pos_ in ['NOUN','ADJ','ADV','VERB'] and not tok.is_stop]
  tokenlist.append(tokens)

In [ ]:
# Import the dictionary builder
from gensim.corpora.dictionary import Dictionary

# Import the TfidfModel from Gensim
from gensim.models.tfidfmodel import TfidfModel

# Just like before, we import the model
from gensim.models.lsimodel import LsiModel

# Tooling to map between corpus (gensim) and matrix - more general
from gensim.matutils import corpus2csc, corpus2dense

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Generate a dictionary and filter
dictionary = Dictionary(tokenlist)
dictionary.filter_extremes(no_below=5, no_above=0.2)

In [ ]:
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(word_tokenize(doc.lower())) for doc in data['Abstract']]

In [ ]:
# Create and fit a new TfidfModel using the corpus: tfidf
tfidf = TfidfModel(corpus)

In [ ]:
# transform corpus to TFIDF
corpus_tfidf = tfidf[corpus]

In [ ]:
# Training the LSI model
model_lsi = LsiModel(corpus_tfidf, num_topics = 300, id2word=dictionary)

In [ ]:
model_lsi.print_topics()

In [ ]:
# Generating the corpus train & test

corpus_lsi = model_lsi[corpus_tfidf]

In [ ]:
# turn into matrix
corpus_lsi_matrix = corpus2dense(corpus_lsi, 300 )

/usr/local/lib/python3.7/dist-packages/gensim/matutils.py:502: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


In [ ]:
corpus_lsi_matrix = corpus_lsi_matrix.T

# Inspect simple topic model with LSI

In [ ]:
# Nothing new here
from sklearn.cluster import KMeans
clusterer = KMeans(n_clusters = 4)
clusterer.fit(corpus_lsi_matrix)

KMeans(n_clusters=4)

In [ ]:
# Let's check out the topics by getting "top-tfidf" for the different clusters (and we need to transponse)
tfidf_matrix = corpus2dense(corpus_tfidf, len(dictionary)).T

/usr/local/lib/python3.7/dist-packages/gensim/matutils.py:502: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


In [ ]:
# write cluster-numbers into our data
data['cluster'] = clusterer.labels_

In [ ]:
# Let's loop it

for i in set(clusterer.labels_):
  cluster_index = data[data['cluster'] == i].index
  topk = np.flip(np.argsort(np.sum(tfidf_matrix[cluster_index,:], axis=0)))[:10]

  print(str(i) + str([dictionary[x] for x in topk]))

0['language', 'public', 'web', 'digital', 'new', 'text', 'cultural', 'development', 'approach', 'content']
1['learning', 'online', 'students', 'perceived', 'knowledge', 'model', 'mobile', 'presence', 'group', 'users']
2['species', 'urban', 'landscape', 'habitat', 'forest', 'diversity', 'plant', 'bird', 'biodiversity', 'vegetation']
3['teams', 'team', 'virtual', 'members', 'task', 'collaboration', 'tools', 'performance', 'trust', 'teamwork']


In [ ]:
!pip install -q annoy

     |████████████████████████████████| 646 kB 4.8 MB/s 


# Run LDA model

In [ ]:
from gensim.models import LdaMulticore

In [ ]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=4, workers = 4, passes=10)

In [ ]:
lda_model.print_topics(-1)

[(0,
  '0.010*"©" + 0.007*"model" + 0.006*"perceived" + 0.006*"users" + 0.006*"online" + 0.004*"task" + 0.004*"virtual" + 0.004*"support" + 0.004*"mobile" + 0.004*"findings"'),
 (1,
  '0.036*"species" + 0.013*"urban" + 0.012*"landscape" + 0.010*"habitat" + 0.010*"diversity" + 0.009*"forest" + 0.008*"©" + 0.007*"areas" + 0.006*"science+business" + 0.006*"plant"'),
 (2,
  '0.022*"learning" + 0.015*"students" + 0.010*"©" + 0.010*"knowledge" + 0.006*"education" + 0.005*"content" + 0.004*"system" + 0.004*"management" + 0.004*"systems" + 0.004*"university"'),
 (3,
  '0.011*"©" + 0.006*"new" + 0.005*"development" + 0.005*"cultural" + 0.004*"digital" + 0.004*"visual" + 0.004*"online" + 0.004*"work" + 0.004*"approach" + 0.004*"culture"')]

In [ ]:
from annoy import AnnoyIndex

In [ ]:
f = 300

t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

for i in range(len(corpus_lsi_matrix)):
    t.add_item(i, corpus_lsi_matrix[i])

In [ ]:
t.build(10)

True

In [ ]:
data['Abstract'][0]

'The online game market is highly popular, resulting in fierce competition among game makers and prompting them to find ways to strengthen gamer loyalty. Online gamers enjoy interacting, socializing, and building quickly formed relationships, i.e., swift guanxi, which greatly rely on games as media through which to exchange rich messages. However, it is not known how these aspects foster gamer loyalty. To fill this gap, we examine how media richness aspects form gamers’ perceptions of presence, interactivity, and swift guanxi, which help foster gamer loyalty. We used a total of 1350 online gamers’ responses to test our model. The results show that two of the four aspects of media richness effectively enhance presence, interactivity, and swift guanxi, which in turn strengthen gamer loyalty. We extend media richness theory by identifying some novel consequences of media richness, i.e., presence, interactivity, and swift guanxi, and establishing their interrelationships. We also ascertain

In [ ]:
t.get_nns_by_item(0, 10)

[0, 190, 350, 125, 122, 1062, 543, 518, 417, 153]

In [ ]:
data['Abstract'][125]

'Purpose: Previous studies have attempted to address online uncertainties from the relationship marketing perspective. The purpose of this paper is to argue that the integration of media richness theory (MRT) and cognitive fit theory (CFT) can contribute a new perspective in addressing consumers’ transaction-specific uncertainties in online retailing. Design/methodology/approach: On the basis of MRT and CFT, a research model was developed by correlating online channel media richness (OCMR), online–offline information integration (OOII), information privacy concern, perceived deception and online loyalty. The model was empirically examined based on survey data collected from 258 multi-channel consumers in China. Findings: An analysis of structural equation model showed that OCMR is negatively associated with information privacy concern and perceived deception but is not significant to online loyalty. Information privacy concern has a negative influence on online loyalty, but the effect 

Even simpler with SpaCy embeddings

In [ ]:
import numpy as np

In [ ]:
# Let's tokenize all abstracts and get a filtered list of tokens
vectors = []
for doc in nlp.pipe(data['Abstract'], disable=["parser", "ner"]):
  vectors.append(doc.vector)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
f = 96

t_spacy = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

for i in range(len(vectors)):
    t_spacy.add_item(i, vectors[i])

In [ ]:
t_spacy.build(50)

True

In [ ]:
t_spacy.get_nns_by_item(0, 10)

[0, 1095, 286, 110, 719, 280, 542, 282, 974, 1123]

In [ ]:
t.get_nns_by_item(0, 10)

[0, 190, 350, 125, 122, 1062, 543, 518, 417, 153]

In [ ]:
data['Abstract'][286]

'Social media enables the performative actions needed for celebrities to build and maintain audiences. Platforms like Twitter mediate identity construction and interaction with fans while enabling environments that are co-constructed by celebrities, fans, and the platform itself. We use the theoretical lens of media richness to study the ways that different types of celebrities enact “micro-celebrity” by mapping three richness dimensions (contextual, interactional, and informational) into groupings of Twitter’s affordances. Utilizing crowdsourcing and regression analysis, we systematically weigh each affordance and generate richness scores for each dimension, for each tweet. Using these richness scores, we find that performance of different types of celebrity requires different affordance mixtures, and that these mixtures reflect differences in the environments within which celebrities operate. Our research contributes to work at the intersection of Twitter affordances and celebrity st